In [1]:
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_core.documents import Document
import os
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import ChatOllama

/Users/gen2/miniconda3/envs/agent/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain_ollama.embeddings import OllamaEmbeddings
embeddings_model = OllamaEmbeddings(
    model="bge-m3:567m"
)

In [3]:
chroma_db = Chroma( 
    embedding_function=embeddings_model,
    collection_name="bible",
    persist_directory='./vectordb2/'
)


In [4]:
retriever = chroma_db.as_retriever()

In [5]:
model = ChatOllama(model="ministral-3:8b")

In [6]:
from langchain.prompts import PromptTemplate

template = """
당신은 성경 전문가입니다. 아래 규칙을 따르세요.
먼저 제공된 Context에서 답을 찾으십시오.
Context에 답이 없거나 불충분하면, 당신의 일반 지식(사전학습 지식)을 바탕으로 답하십시오. 이 경우 "일반 지식"임을 명시하고, 가능하면 출처나 근거(구절, 통상적 학설)를 덧붙이십시오.
확실하지 않다면 불확실하다고 밝히고, 추가 검색이 필요한 부분을 제안하십시오.
Context
{context}

Question
{question}

Answer
"""

prompt = PromptTemplate.from_template(template)


In [7]:
chain = (
    {'context' : retriever, 'question' : RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [8]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(

llm=model,
chain_type="stuff",
retriever=retriever,
chain_type_kwargs={"prompt": prompt} # 위 혼합모드 프롬프트

)

In [9]:
q = "에녹은 어디서 나오나?"

In [10]:
rt = qa.invoke(q)

ResponseError: model 'ministral-3:7b' not found (status code: 404)

In [ ]:
print(rt['result'])